# Part 1: Scraping Wikipedia for PostalCode, Borough, Neighborhood and creating dataframe

In [105]:
#general imports
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests as r
from tabulate import tabulate as tab
import numpy as np
print("Complete")


Complete


In [23]:
#create json list of wiki table records
wikitor = r.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = bs(wikitor.content,'lxml')
table = soup.find_all('table')[0]
tordf = pd.read_html(str(table))
wikitable = tordf[0].to_json(orient='records')
wikitable


'[{"Postal Code":"M1A","Borough":"Not assigned","Neighbourhood":"Not assigned"},{"Postal Code":"M2A","Borough":"Not assigned","Neighbourhood":"Not assigned"},{"Postal Code":"M3A","Borough":"North York","Neighbourhood":"Parkwoods"},{"Postal Code":"M4A","Borough":"North York","Neighbourhood":"Victoria Village"},{"Postal Code":"M5A","Borough":"Downtown Toronto","Neighbourhood":"Regent Park, Harbourfront"},{"Postal Code":"M6A","Borough":"North York","Neighbourhood":"Lawrence Manor, Lawrence Heights"},{"Postal Code":"M7A","Borough":"Downtown Toronto","Neighbourhood":"Queen\'s Park, Ontario Provincial Government"},{"Postal Code":"M8A","Borough":"Not assigned","Neighbourhood":"Not assigned"},{"Postal Code":"M9A","Borough":"Etobicoke","Neighbourhood":"Islington Avenue, Humber Valley Village"},{"Postal Code":"M1B","Borough":"Scarborough","Neighbourhood":"Malvern, Rouge"},{"Postal Code":"M2B","Borough":"Not assigned","Neighbourhood":"Not assigned"},{"Postal Code":"M3B","Borough":"North York","Ne

In [30]:
#convert to dataframe    
toronto = pd.read_json(wikitable)
toronto.head()
toronto.shape

(180, 3)

In [48]:
#drop rows for unassigned boroughs
toronto = toronto[toronto.Borough != 'Not assigned']
toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [34]:
#set neighbourhood = borough where neighbourhood is not assigned
toronto.loc[toronto['Neighbourhood'] == 'Not assigned']

#all neighbourhoods are assigned already

,Postal Code,Borough,Neighbourhood


In [35]:
toronto.shape

(103, 3)

# Part 2: Adding Lat+Lon to Dataframe from csv

In [38]:
#downlod csv and read into dataframe
!wget -q -O 'lat_lon.csv' https://cocl.us/Geospatial_data
latlon = pd.read_csv('lat_lon.csv')
latlon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
#merge lat+lon into toronto dataframe
tor_ll = pd.merge(toronto,latlon,on = 'Postal Code', how = 'left')
tor_ll.head()
tor_ll.shape

(103, 5)

In [45]:
#check for any NaN Lat or Lon values, all are loaded
tor_ll[tor_ll.isna().any(axis=1)]

,Postal Code,Borough,Neighbourhood,Latitude,Longitude


In [50]:
print(tor_ll.loc[tor_ll['Postal Code'] == 'M1B'])
print(toronto.loc[toronto['Postal Code'] == 'M1B'])

  Postal Code      Borough   Neighbourhood   Latitude  Longitude
6         M1B  Scarborough  Malvern, Rouge  43.806686 -79.194353
  Postal Code      Borough   Neighbourhood
9         M1B  Scarborough  Malvern, Rouge


In [51]:
tor_ll

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


# Part 3: Exploring and Clustering Neighbourhoods

In [83]:
# The code was removed by Watson Studio for sharing.

In [66]:
#import tools

from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
! pip install folium==0.5.0
import folium
!pip install geocoder
import geocoder as geo

print('complete')

complete


In [78]:
#get toronto lat+lon
user = Nominatim(user_agent='tor_exp')
ll = user.geocode('Toronto, ON')
latitude = ll.latitude
longitude = ll.longitude
print(latitude)
print(longitude)
ll


43.6534817
-79.3839347


Location(Toronto, Golden Horseshoe, Ontario, Canada, (43.6534817, -79.3839347, 0.0))

In [81]:
#create map with unclustered data

tormap = folium.Map(location=[latitude, longitude], zoom_start=10)
for lat, lng, borough, neighborhood in zip(tor_ll['Latitude'], tor_ll['Longitude'], tor_ll['Borough'], tor_ll['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tormap) 
tormap

**Cluster boroughs that contain 'Toronto' to see top 10 venue categories per borough**

In [90]:
#create dataframe with only 'Toronto' boroughs
torbor = tor_ll[tor_ll['Borough'].str.contains("Toronto")]
print(torbor.shape)
torbor.head()

(39, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [113]:
#list by neighbourhood
torneigh = torbor.assign(neigh1=torbor['Neighbourhood'].str.split(', ')).explode('neigh1')
torneigh

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,neigh1
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Regent Park
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Harbourfront
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Queen's Park
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Ontario Provincial Government
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Garden District
...,...,...,...,...,...,...
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,First Canadian Place
97,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.648429,-79.382280,Underground city
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,Business reply mail Processing Centre


In [117]:
#get all venues for 'Toronto' boroughs
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
    
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
  
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [118]:
tor_venues = getNearbyVenues(names=torneigh['neigh1'],latitudes=torneigh['Latitude'],longitudes=torneigh['Longitude'])

Regent Park
Harbourfront
Queen's Park
Ontario Provincial Government
Garden District
Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond
Adelaide
King
Dufferin
Dovercourt Village
Harbourfront East
Union Station
Toronto Islands
Little Portugal
Trinity
The Danforth West
Riverdale
Toronto Dominion Centre
Design Exchange
Brockton
Parkdale Village
Exhibition Place
India Bazaar
The Beaches West
Commerce Court
Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
Forest Hill Road Park
High Park
The Junction South
North Toronto West
Lawrence Park
The Annex
North Midtown
Yorkville
Parkdale
Roncesvalles
Davisville
University of Toronto
Harbord
Runnymede
Swansea
Moore Park
Summerhill East
Kensington Market
Chinatown
Grange Park
Summerhill West
Rathnelly
South Hill
Forest Hill SE
Deer Park
CN Tower
King and Spadina
Railway Lands
Harbourfront West
Bathurst Quay
South Niagara
Island airport
Rosedale
Stn A PO Boxes
St. James Tow

In [119]:
print(tor_venues.shape)
tor_venues.head()

(3143, 7)


,Neighbourhood,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Regent Park,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Regent Park,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Regent Park,43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,Regent Park,43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,Regent Park,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [120]:
tor_venues.groupby('Neighbourhood').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Adelaide,95,95,95,95,95,95
Bathurst Quay,16,16,16,16,16,16
Berczy Park,58,58,58,58,58,58
Brockton,24,24,24,24,24,24
Business reply mail Processing Centre,17,17,17,17,17,17
...,...,...,...,...,...,...
Underground city,100,100,100,100,100,100
Union Station,100,100,100,100,100,100
University of Toronto,32,32,32,32,32,32


In [121]:
#Add onehot encoding to see what venue categories are in each borough
tor_onehot = pd.get_dummies(tor_venues[['Venue Category']], prefix="", prefix_sep="")
tor_onehot['Neighbourhood']=tor_venues['Neighbourhood']
fixed_columns = [tor_onehot.columns[-1]] + list(tor_onehot.columns[:-1])
tor_onehot = tor_onehot[fixed_columns]
tor_onehot.head()

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Regent Park,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [123]:
#group on borough name
toronto_groups = tor_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_groups

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Adelaide,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.021053,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.010526,0.00000,0.0,0.00,0.000000
1,Bathurst Quay,0.0625,0.0625,0.0625,0.125,0.125,0.125,0.000000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.00000,0.0,0.00,0.000000
2,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.017241,0.00000,0.0,0.00,0.000000
3,Brockton,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.00000,0.0,0.00,0.041667
4,Business reply mail Processing Centre,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.00000,0.0,0.00,0.058824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Underground city,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.030000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.01,0.010000,0.00000,0.0,0.01,0.000000
72,Union Station,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.05,...,0.0,0.0,0.0,0.0,0.01,0.010000,0.00000,0.0,0.01,0.000000
73,University of Toronto,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.000000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.000000,0.03125,0.0,0.00,0.031250
74,Victoria Hotel,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.040000,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.020000,0.00000,0.0,0.01,0.000000


In [125]:
#create dataframe with top 10 venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
borough_venues_sorted = pd.DataFrame(columns=columns)
borough_venues_sorted['Neighbourhood'] = toronto_groups['Neighbourhood']

for ind in np.arange(toronto_groups.shape[0]):
    borough_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_groups.iloc[ind, :], num_top_venues)

borough_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adelaide,Coffee Shop,Café,Restaurant,Gym,Thai Restaurant,Clothing Store,Bakery,Deli / Bodega,American Restaurant,Bookstore
1,Bathurst Quay,Airport Lounge,Airport Service,Airport Terminal,Airport,Bar,Coffee Shop,Rental Car Location,Sculpture Garden,Boat or Ferry,Boutique
2,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,Jazz Club,Shopping Mall
3,Brockton,Café,Breakfast Spot,Coffee Shop,Yoga Studio,Grocery Store,Pet Store,Performing Arts Venue,Office,Nightclub,Italian Restaurant
4,Business reply mail Processing Centre,Light Rail Station,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,Restaurant,Burrito Place,Brewery,Skate Park


In [130]:
#k-means clustering

kclusters = 5

toronto_grouped_clustering = toronto_groups.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([4, 0, 4, 4, 0, 0, 4, 4, 4, 4], dtype=int32)

In [133]:
#borough_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
borough_venues_sorted
tor_merge = torneigh
tor_merge = tor_merge.join(borough_venues_sorted.set_index('Neighbourhood'),on='neigh1')
tor_merge.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,neigh1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Regent Park,4,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Shoe Store,Brewery,Restaurant
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,Harbourfront,4,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Shoe Store,Brewery,Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Queen's Park,4,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Portuguese Restaurant,Park,Nightclub,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,Ontario Provincial Government,4,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Portuguese Restaurant,Park,Nightclub,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,Garden District,4,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Hotel,Café,Bubble Tea Shop,Italian Restaurant,Theater


In [134]:
#mapping clusters with folium
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tor_merge['Latitude'], tor_merge['Longitude'], tor_merge['Neighbourhood'], tor_merge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Observations

By far the most common grouping was in cluster 4. Clusters 1,2 & 3 were only represented once a piece, and cluster 0 was represented 5 times. The predominate industry for each cluster can be seen below:

In [140]:
econ = pd.DataFrame({'Cluster': [0,1,2,3,4],'Predominate Industry':['Travel/Commuting','Home Service','Exercise/Shopping','Luxury','Urban Life']})
econ

,Cluster,Predominate Industry
0,0,Travel/Commuting
1,1,Home Service
2,2,Exercise/Shopping
3,3,Luxury
4,4,Urban Life


In [139]:
tor_merge.loc[tor_merge['Cluster Labels'] == 4, tor_merge.columns[[1] + list(range(5, tor_merge.shape[1]))]]

,Borough,neigh1,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,Regent Park,4,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Shoe Store,Brewery,Restaurant
2,Downtown Toronto,Harbourfront,4,Coffee Shop,Park,Bakery,Pub,Breakfast Spot,Café,Theater,Shoe Store,Brewery,Restaurant
4,Downtown Toronto,Queen's Park,4,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Portuguese Restaurant,Park,Nightclub,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
4,Downtown Toronto,Ontario Provincial Government,4,Coffee Shop,Sushi Restaurant,Yoga Studio,Distribution Center,Portuguese Restaurant,Park,Nightclub,Mexican Restaurant,Japanese Restaurant,Italian Restaurant
9,Downtown Toronto,Garden District,4,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Hotel,Café,Bubble Tea Shop,Italian Restaurant,Theater
9,Downtown Toronto,Ryerson,4,Clothing Store,Coffee Shop,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Hotel,Café,Bubble Tea Shop,Italian Restaurant,Theater
15,Downtown Toronto,St. James Town,4,Coffee Shop,Café,Gastropub,Restaurant,Italian Restaurant,Pizza Place,Bakery,Park,American Restaurant,Cocktail Bar
20,Downtown Toronto,Berczy Park,4,Coffee Shop,Cocktail Bar,Beer Bar,Farmers Market,Seafood Restaurant,Bakery,Restaurant,Cheese Shop,Jazz Club,Shopping Mall
24,Downtown Toronto,Central Bay Street,4,Coffee Shop,Café,Sandwich Place,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Discount Store,Ice Cream Shop,Indian Restaurant
25,Downtown Toronto,Christie,4,Grocery Store,Café,Park,Candy Store,Italian Restaurant,Restaurant,Baby Store,Nightclub,Coffee Shop,Athletics & Sports
